In [0]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
from time import time

In [0]:
import numpy as np
import pandas as pd

In [0]:
df = pd.read_csv('Datos_procedimientos_nuevo.csv', delimiter=";")

# Obtenemos una lista de los valores de la columna Case
cases = sorted(list(df['Case'].values))

# Eliminamos los repetidos

patients = list()
for case in cases:
    if case not in patients:
        patients.append(case)

# Se categorizan los procedimientos 
procedure = []
        
for i, row in df.iterrows():
    if row['Procedure'][0] == "0":
        if row['LOS'] > 0 or row['LOS'] <= 1: 
            procedure.append("0A")
        elif row['LOS'] > 1 or row['LOS'] <= 2: 
            procedure.append("0B")
        elif row['LOS'] > 2 or row['LOS'] <= 5: 
            procedure.append("0C")
        elif row['LOS'] > 5 or row['LOS'] <= 10: 
            procedure.append("0D")
        elif row['LOS'] > 10 or row['LOS'] <= 25: 
            procedure.append("0E")
        elif row['LOS'] > 25: 
            procedure.append("0F")    
    else:
        procedure.append(row['Procedure'][0])
    

df['Procedure'] = procedure

# Funcion que formatea los datos en un string unido por guiones
def formatter(lista):
    return "-".join(lista)

filas = list()

for i in range(len(patients)):
    # Se filtra la base de datos solo con los datos para el paciente i
    filtro = df[df['Case'] == patients[i]]
    
    # Se obtienen sus procedimientos
    procedures = set(filtro['Procedure'].values)
    
    # Si se realizo un procedimiento mas, se agrupan de tal forma
    if len(procedures) > 1:
        procedure = formatter(sorted(list(procedures)))
        
    else:
        procedure = list(procedures)[0]
    
    case = patients[i]
    los = max(list(filtro['LOS'].values))
    row = (case, procedure, los)
    
    filas.append(row)

process_df = pd.DataFrame(filas, columns=["Case", "Procedure", "LOS"])

In [8]:
# Se importan los datos para ambas bases de datos
diagnose_df = pd.read_excel("Datos_diagnosticos.xlsx") # Datos de diagnostico de pacientes
#process_df = pd.read_csv('grouped_procedures.csv', delimiter=";") # Datos de procedimientos

process_df = process_df.rename({"Case": "CASE"}, axis="columns")
process_df = process_df.rename({"LOS (days)": "LOS"}, axis="columns")

process_df

,CASE,Procedure,LOS
0,13872110,0A,3
1,14035188,0A,3
2,14085514,0A,1
3,14111667,0A,3
4,14111831,0A,2
...,...,...,...
11946,15056499,4-5,1
11947,15056507,4,1
11948,15056517,4,1
11949,15057405,0A,1


In [0]:
# Guardamos el diagnostico de acuerdo a su primera letra
diagnose = [d[0] for d in diagnose_df["Diagnosis"]]

diagnose_df["Diagnosis"] = diagnose

In [10]:
diagnose_df

,CASE,Seq,PrincSec,Diagnosis
0,13872110,1,P,E
1,13872110,1,S,Z
2,13872110,1,S,D
3,14035188,1,P,J
4,14035188,1,S,I
...,...,...,...,...
110055,15057405,2,S,D
110056,15057706,1,P,O
110057,15057706,2,P,O
110058,15057706,2,S,Z


In [0]:
# Se realiza un join entre ambas bases de datos
joined = pd.merge(process_df, diagnose_df, on="CASE")

In [12]:
print(len(joined["CASE"]))
joined.head(10)

110060


,CASE,Procedure,LOS,Seq,PrincSec,Diagnosis
0,13872110,0A,3,1,P,E
1,13872110,0A,3,1,S,Z
2,13872110,0A,3,1,S,D
3,14035188,0A,3,1,P,J
4,14035188,0A,3,1,S,I
5,14035188,0A,3,1,S,E
6,14035188,0A,3,1,S,Z
7,14035188,0A,3,1,S,D
8,14085514,0A,1,1,P,S
9,14085514,0A,1,1,S,V


In [13]:
# Eliminamos la columna CASE
data = joined.drop("CASE", axis=1)

data

,Procedure,LOS,Seq,PrincSec,Diagnosis
0,0A,3,1,P,E
1,0A,3,1,S,Z
2,0A,3,1,S,D
3,0A,3,1,P,J
4,0A,3,1,S,I
...,...,...,...,...,...
110055,0A,1,2,S,D
110056,1,0,1,P,O
110057,1,0,2,P,O
110058,1,0,2,S,Z


In [0]:
diagnosis_values = list(data['Diagnosis'].unique())
princsec_values = list(data['PrincSec'].unique())
procedure_values = list(data['Procedure'].unique())

In [15]:
len(procedure_values)

103

In [0]:
from sklearn.preprocessing import KBinsDiscretizer

In [0]:
# Tomamos la columna de LOS y la discretizamos en 10 bins

# Primero se debe transformar los valores a floats
# Se transforman los valores a un array de (n_los, 1), 
# donde n_los es la cant de datos y es una colmna
normLOS = np.asarray(data['LOS'].astype(float)).reshape(-1, 1)


In [0]:
# Se setea el modelo; notar que se utiliza el algoritmo kMeans para encontrar clusters de la variable LOS
# De acuerdo a su valor mas cercano al promedio de cada bin
model = KBinsDiscretizer(n_bins=19, encode='ordinal', strategy='kmeans')

In [19]:
# Se fitean los datos de acuerdo al modelo
print(model.fit(normLOS))

# Intervalos
bins = model.bin_edges_[0]
print(bins)

KBinsDiscretizer(encode='ordinal', n_bins=19, strategy='kmeans')
[  0.           2.70764581   6.51323177  12.84380922  22.28171972
  34.01343311  47.34669905  62.01508413  77.11191794  93.1776969
 107.95865558 122.61590493 136.5782083  149.20080446 170.00386905
 192.06501035 205.5298913  215.89008621 241.57758621 262.        ]


In [0]:
# Transformamos los bins a intervalos
def makeIntervals(bins):
    intervals = list()
    
    for i in range(len(bins)-1):
        intervals.append((bins[i], bins[i+1]))
    return intervals

bins = makeIntervals(bins)

In [0]:
# Se define una funcion que retorna un arreglo con el intervalo correspondiente para cada dato
def getInterval(dato, intervalos):
    for inter in intervalos:
        if inter[0] <= dato <= inter[1]:
            return inter
        
def getBinnedData(array, intervalos):
    
    binned_data = list()
    for dato in array:
        intervalo = getInterval(dato, intervalos)
        binned_data.append(intervalo)
        
    return binned_data
        
        

In [0]:
# Se obtienen los datos 'categorizados'
catLOS = model.transform(normLOS)

In [23]:
catLOS

array([[1.],
       [1.],
       [1.],
       ...,
       [0.],
       [0.],
       [0.]])

In [0]:
# Para volver atrás, se realiza lo siguiente

# Cada valor se transforma a la media de las dos cotas del intervalo
catLOS_inverse = model.inverse_transform(catLOS)

In [0]:
# Se agrega la columna binned_LOS que contiene información del 'cluster' en que esta el LOS
data['Binned_LOS'] = pd.DataFrame(catLOS)

# Se agrega la columna inversed_binned_LOS que muestra la transformación inversa del cluster al dato;
# Este valor es un promedio de las cotas de cada intervalo en el que se muestra el dato
data['inversed_binned_LOS'] = pd.DataFrame(catLOS_inverse)

In [26]:
data

,Procedure,LOS,Seq,PrincSec,Diagnosis,Binned_LOS,inversed_binned_LOS
0,0A,3,1,P,E,1.0,4.610439
1,0A,3,1,S,Z,1.0,4.610439
2,0A,3,1,S,D,1.0,4.610439
3,0A,3,1,P,J,1.0,4.610439
4,0A,3,1,S,I,1.0,4.610439
...,...,...,...,...,...,...,...
110055,0A,1,2,S,D,0.0,1.353823
110056,1,0,1,P,O,0.0,1.353823
110057,1,0,2,P,O,0.0,1.353823
110058,1,0,2,S,Z,0.0,1.353823


# Random Forest para clasificar los datos

Ahora que tenemos la base de datos con variables categóricas (la variable _LOS_ fue discretizada mediante *kMeans*), se procederá a utilizar el modelo de aprendizaje supervisado *RANDOM FOREST*, el cual permite establecer predicciones del intervalo en el que estará el LOS, posterior a que se le ha realizado un entrenamiento. 

## Preprocesamiento de los datos

La distribución de los tiempos de estadía de los pacientes está cargada hacia valores de LOS entre 0 y 25 días, concentrando un 90 % de los datos en tal intervalo, aproximadamente. Por consecuencia, nuestra base de datos está desbalanceada, por lo que debemos realizar una *partición estratificada* de los datos, a modo de obtener una proporciones similares de cada intervalo de LOS, tanto en el training set como en el testing set.

Utilizamos la librería de **scikit learn** para realizar lo anterior: 

In [0]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing

In [28]:
# Para que el modelo funcione, se presentan los datos como si fueran numeros enteros
le = preprocessing.LabelEncoder()

categories = ["Diagnosis", "PrincSec", "Procedure"]
dictCategories = {}

for cat in categories:
    
    le.fit(data[cat])
    print(f"Categoria: {cat} => {le.classes_}")
    print(f"Transformacion: {[i for i in range(len(le.classes_))]}")
    print()
    
    dictCategories[cat] = le.transform(data[cat])
    
dictCategories['Seq'] = np.array(data['Seq'])

# Se muestran los datos categoricos representados como enteros
print()
print("Datos categoricos como enteros:")
print(dictCategories)

Categoria: Diagnosis => ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R'
 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z']
Transformacion: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]

Categoria: PrincSec => ['P' 'S']
Transformacion: [0, 1]

Categoria: Procedure => ['0A' '0A-1' '0A-1-2' '0A-1-3' '0A-1-3-4' '0A-1-4' '0A-2' '0A-2-3'
 '0A-2-3-5' '0A-2-3-5-8-F' '0A-2-3-8' '0A-2-3-B-H' '0A-2-8' '0A-3'
 '0A-3-4' '0A-3-4-5' '0A-3-4-5-6-8-B' '0A-3-4-5-B' '0A-3-4-5-B-C'
 '0A-3-4-8-B' '0A-3-4-B' '0A-3-4-F' '0A-3-5' '0A-3-5-6' '0A-3-5-8'
 '0A-3-5-8-B' '0A-3-5-8-C-D' '0A-3-5-8-D' '0A-3-5-B' '0A-3-5-C' '0A-3-6'
 '0A-3-6-B' '0A-3-8' '0A-3-8-B-H' '0A-3-8-F' '0A-3-B' '0A-3-B-D'
 '0A-3-B-F' '0A-3-C' '0A-3-C-D' '0A-3-D' '0A-3-F' '0A-3-H' '0A-4' '0A-4-5'
 '0A-4-5-B' '0A-4-5-B-H' '0A-4-B' '0A-4-B-H' '0A-5' '0A-5-8' '0A-5-8-F'
 '0A-5-B' '0A-5-C' '0A-5-F' '0A-5-X' '0A-6' '0A-8' '0A-B' '0A-B-F' '0A-C'
 '0A-D' '0A-F' '0A-H' '1' '1-3' '1-3-4' '1-4' '2' 

In [0]:
# Dividimos nuestra base de datos en las features y el target (LOS)

features = pd.DataFrame(dictCategories)

# Utilizamos binned_LOS como nuestra variable a predecir, ya que discretizamos el LOS
target = data['Binned_LOS']

In [0]:
# Realizamos la partición estratificada; consideramos la proporcion 80-20 entre
# datos de entrenamiento y datos de test
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2,
                                                    random_state=1, stratify=target)

In [31]:
X_test

,Diagnosis,PrincSec,Procedure,Seq
28080,10,0,0,2
30141,25,1,13,1
71872,3,1,0,1
23291,4,1,0,1
88143,19,0,0,1
...,...,...,...,...
41571,25,1,0,2
19699,18,1,0,1
13968,25,1,0,1
63131,19,1,0,1


In [0]:
# Definimos una función que permite entrenar los datos con el RandomForest

def fitRandomForest(X, y, seed=0):
    '''
    n_estimators: numero de arboles de clasificacion en el algoritmo
    max_depth: maxima profundidad en el árbol
    class_weight: pesos asignados a cada clase a predecir
    '''
    clf = RandomForestClassifier(n_estimators=75, max_depth=10, 
                                 class_weight='balanced' , random_state=seed)
    trained = clf.fit(X, y)
    
    return trained

def fitDecisionTree(X, y, seed=0):
    
    clf = DecisionTreeClassifier(criterion="entropy",
                                max_depth=10,
                                class_weight="balanced",
                                min_samples_leaf= 10,
                                max_leaf_nodes=5,
                                random_state=seed)
    trained = clf.fit(X, y)
    
    return trained

# Funcion que entrega una prediccion para los datos de prueba 
def prediction(X_test, trained):
    y_pred = trained.predict(X_test)
    return y_pred

In [0]:
# Aplicamos lo anterior a nuestra base de datos

trained = fitDecisionTree(X_train, y_train, seed=1)
y_pred = prediction(X_test, trained)


In [34]:
y_pred

array([ 1., 17.,  1., ...,  1.,  1.,  0.])

In [0]:
# Se transforman los datos al valor promedio del cluster
y_pred_reshaped = np.asarray(y_pred).reshape(-1, 1)
y_pred_ = model.inverse_transform(y_pred_reshaped).reshape(1, len(y_pred_reshaped))
y_pred_ = list(y_pred_[0])

# Se obtiene el valor real de los LOS en la base de datos de test
y_test_real = [data['LOS'].values[i] for i in X_test.index.tolist()]

In [0]:
# Se calcula el intervalo
y_pred_interval = getBinnedData(y_pred_, bins)

In [37]:
print(y_test)

28080    0.0
30141    0.0
71872    0.0
23291    0.0
88143    2.0
        ... 
41571    1.0
19699    1.0
13968    2.0
63131    2.0
42621    3.0
Name: Binned_LOS, Length: 22012, dtype: float64


In [38]:
comparacion = list(zip(y_test_real, y_pred_))

for i, j in comparacion:
    print(f"Dato de prueba: {i}; Dato predicho: {j}")

Se han truncado las últimas 5000 líneas del flujo de salida.
Dato de prueba: 23; Dato predicho: 1.3538229051619246
Dato de prueba: 5; Dato predicho: 4.610438790554548
Dato de prueba: 2; Dato predicho: 4.610438790554548
Dato de prueba: 1; Dato predicho: 4.610438790554548
Dato de prueba: 1; Dato predicho: 1.3538229051619246
Dato de prueba: 1; Dato predicho: 1.3538229051619246
Dato de prueba: 4; Dato predicho: 4.610438790554548
Dato de prueba: 103; Dato predicho: 198.7974508281577
Dato de prueba: 5; Dato predicho: 1.3538229051619246
Dato de prueba: 21; Dato predicho: 4.610438790554548
Dato de prueba: 7; Dato predicho: 198.7974508281577
Dato de prueba: 1; Dato predicho: 4.610438790554548
Dato de prueba: 33; Dato predicho: 228.7338362068968
Dato de prueba: 6; Dato predicho: 228.7338362068968
Dato de prueba: 4; Dato predicho: 4.610438790554548
Dato de prueba: 16; Dato predicho: 4.610438790554548
Dato de prueba: 6; Dato predicho: 4.610438790554548
Dato de prueba: 4; Dato predicho: 1.353822905

In [50]:
categories = ['Diagnosis', 'PrincSec', 'Procedure']

dict_X_test = {}

for cat in categories:
    le.fit(data[cat])
    
    print(le.classes_)
    
    dict_X_test[cat] = list(le.inverse_transform(X_test[cat]))
    
    
    
dict_X_test['Seq'] = X_test['Seq']
dict_X_test['LOS'] = y_test_real
dict_X_test['LOS_predicted'] = y_pred_
dict_X_test['LOS_predicted_bin'] = y_pred_interval

X_test_with_prediction = pd.DataFrame(dict_X_test)

X_test_with_prediction.head(20)

['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R'
 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z']
['P' 'S']
['0A' '0A-1' '0A-1-2' '0A-1-3' '0A-1-3-4' '0A-1-4' '0A-2' '0A-2-3'
 '0A-2-3-5' '0A-2-3-5-8-F' '0A-2-3-8' '0A-2-3-B-H' '0A-2-8' '0A-3'
 '0A-3-4' '0A-3-4-5' '0A-3-4-5-6-8-B' '0A-3-4-5-B' '0A-3-4-5-B-C'
 '0A-3-4-8-B' '0A-3-4-B' '0A-3-4-F' '0A-3-5' '0A-3-5-6' '0A-3-5-8'
 '0A-3-5-8-B' '0A-3-5-8-C-D' '0A-3-5-8-D' '0A-3-5-B' '0A-3-5-C' '0A-3-6'
 '0A-3-6-B' '0A-3-8' '0A-3-8-B-H' '0A-3-8-F' '0A-3-B' '0A-3-B-D'
 '0A-3-B-F' '0A-3-C' '0A-3-C-D' '0A-3-D' '0A-3-F' '0A-3-H' '0A-4' '0A-4-5'
 '0A-4-5-B' '0A-4-5-B-H' '0A-4-B' '0A-4-B-H' '0A-5' '0A-5-8' '0A-5-8-F'
 '0A-5-B' '0A-5-C' '0A-5-F' '0A-5-X' '0A-6' '0A-8' '0A-B' '0A-B-F' '0A-C'
 '0A-D' '0A-F' '0A-H' '1' '1-3' '1-3-4' '1-4' '2' '2-3' '3' '3-4' '3-4-5'
 '3-4-B' '3-4-F' '3-5' '3-5-6' '3-5-B' '3-5-H' '3-6' '3-B' '3-B-F' '3-D'
 '3-F' '3-F-H' '4' '4-5' '4-5-B' '4-B' '5' '5-6' '5-B' '6' '8' 'B' 'B-F-H'
 'B-H' 'C' 'D' 'F' 'F-H' 'G' 'H']


,Diagnosis,PrincSec,Procedure,Seq,LOS,LOS_predicted,LOS_predicted_bin
28080,K,P,0A,2,2,4.610439,"(2.707645810323849, 6.513231770785247)"
30141,Z,S,0A-3,1,1,228.733836,"(215.8900862068969, 241.57758620689665)"
71872,D,S,0A,1,1,4.610439,"(2.707645810323849, 6.513231770785247)"
23291,E,S,0A,1,2,4.610439,"(2.707645810323849, 6.513231770785247)"
88143,T,P,0A,1,7,4.610439,"(2.707645810323849, 6.513231770785247)"
94991,D,S,4,1,1,1.353823,"(0.0, 2.707645810323849)"
31042,I,S,0A-3,2,7,228.733836,"(215.8900862068969, 241.57758620689665)"
82429,T,S,0A,2,55,4.610439,"(2.707645810323849, 6.513231770785247)"
108787,C,P,0A-3,2,5,228.733836,"(215.8900862068969, 241.57758620689665)"
50865,Z,S,0A,1,2,4.610439,"(2.707645810323849, 6.513231770785247)"


In [40]:
contador1 = 0
for i, row in X_test_with_prediction.iterrows():
  if row["LOS"] >= row['LOS_predicted_bin'][0] and row["LOS"] <= row['LOS_predicted_bin'][1]:
    contador1 += 1

print(contador1)



6106


Para obtener una métrica de qué tan buena ha sido la predicción, sumamos los errores.

In [0]:
errors = sum(obs[1]["LOS"] - obs[1]["LOS_predicted"] for obs in X_test_with_prediction.iterrows())
abs_errors = sum(abs(obs[1]["LOS"] - obs[1]["LOS_predicted"]) for obs in X_test_with_prediction.iterrows())

In [42]:
print(f"Suma de errores: {errors}")
print(f"Suma de errores absolutos: {abs_errors}")

Suma de errores: -727296.8991848828
Suma de errores absolutos: 937032.8579081022


In [0]:
promedio = abs_errors/len(y_pred_)

In [44]:
print(f"Promedio de errores: {promedio}")

Promedio de errores: 42.5691830777804


In [0]:
suma_var = sum((obs[1]["LOS_predicted"] - promedio)**2 for obs in X_test_with_prediction.iterrows())

In [0]:
varianza = np.sqrt(suma_var/len(y_pred_))

In [47]:
print(f"Varianza: {varianza}")

Varianza: 86.10678164455851


In [48]:
X_test_with_prediction.to_csv(f"datos/prediccion.csv", index=False, header=True)

FileNotFoundError: ignored